In [6]:
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
import rasterio
import rasterio.features
from rasterio.mask import mask
from rasterio.plot import show
import os 
import numpy as np
from rasterstats import zonal_stats
import fiona

In [2]:
# #br = gpd.read_file(r'Z:\- GIS -\IBGE\SC_Municipios_2020\SC_Municipios_2020.shp')
# sc = gpd.read_file(r'D:\GeoBase\IBGE\Municipios_SC_UTM.shp')
# fig,ax = plt.subplots()
# #br.plot(ax=ax)

# #br[br['SIGLA_UF']=='SC'].explore()
# sc[sc['SIGLA_UF']=='SC'].explore('AREA_KM2')

In [12]:
#srtm = rasterio.open(r'Z:\- GIS -\SRTM\SRTM_SC_UTM.tif')
srtm = rasterio.open(r'D:\GeoBase\EPAGRI\SRTM\SRTM_SC_UTM_SIRGAS.tif') #SRTM_SC_UTM_SIRGAS.tif')
srtm.width


18207

In [13]:
srtm.height

12615

In [14]:
{i: dtype for i, dtype in zip(srtm.indexes, srtm.dtypes)}

{1: 'float32'}

In [15]:
srtm.bounds

BoundingBox(left=216916.3754, bottom=6750693.5841, right=763126.3754, top=7129143.5841)

In [16]:
srtm.crs

CRS.from_epsg(31982)

In [17]:
srtm.indexes

(1,)

In [18]:
band1 = srtm.read(1)
band1

array([[-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
        -3.4028235e+38, -3.4028235e+38, -3.4028235e+38],
       [-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
        -3.4028235e+38, -3.4028235e+38, -3.4028235e+38],
       [-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
        -3.4028235e+38, -3.4028235e+38, -3.4028235e+38],
       ...,
       [-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
        -3.4028235e+38, -3.4028235e+38, -3.4028235e+38],
       [-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
        -3.4028235e+38, -3.4028235e+38, -3.4028235e+38],
       [-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
        -3.4028235e+38, -3.4028235e+38, -3.4028235e+38]], dtype=float32)

In [19]:
 affine = srtm.transform

In [20]:
# no data values of the original raster
no_data=srtm.nodata
print(no_data)

-3.4028234663852886e+38


In [ ]:
# change no data values?
# Replace nodata values with a new value (e.g., 0) new_raster_data = np.where(raster_data == nodata_value, 0, raster_data) 

In [10]:
#shape1 = gpd.read_file(r'Z:\- GIS -\IBGE\vegetacao\VG_REG_FITO_UTM_SC.shp')
shape1 = gpd.read_file(r'D:\GeoBase\IBGE\Municipios_SC_UTM.shp')
shape1.crs

<Projected CRS: EPSG:31982>
Name: SIRGAS 2000 / UTM zone 22S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 54°W and 48°W, northern and southern hemispheres, onshore and offshore. In remainder of South America - between 54°W and 48°W, southern hemisphere, onshore and offshore.
- bounds: (-54.0, -54.18, -47.99, 7.04)
Coordinate Operation:
- name: UTM zone 22S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [22]:
shape1.geometry

0    POLYGON ((232290.904 7066073.670, 232301.698 7...
Name: geometry, dtype: geometry

In [13]:
with srtm as src:
    affine = src.transform
    array = src.read(1)
    df_zonal_stats = pd.DataFrame(zonal_stats(shape1, array, affine=affine, stats="count min mean max majority range percentile_10 percentile_90"))

# adding statistics back to original GeoDataFrame
gdf2 = pd.concat([shape1, df_zonal_stats], axis=1) 
gdf2.to_file(r'D:\ENS410064\Dados\atividade03.shp')  

D:\ENS410064\pyEnv\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
D:\ENS410064\pyEnv\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
D:\ENS410064\pyEnv\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
D:\ENS410064\pyEnv\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
D:\ENS410064\pyEnv\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
D:\ENS410064\pyEnv\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_s

If you want to retain the geometries and properties of the input features, you can output a list of geojson features using geojson_out. The features contain the zonal statistics as additional properties:

https://pythonhosted.org/rasterstats/manual.html#zonal-statistics